In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

import datetime as dt

# import mysql.connector as cnt
import sqlite3

import requests
from bs4 import BeautifulSoup

# import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# Firefox leistų sudėtingesnius nustatymus, bet neveikia per SNAP

from selenium import webdriver
from bs4 import BeautifulSoup
import time

print('Paskaita nr. 45, 2024-10-22')
print('Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.')
print('Nuo šios versijos parametrų identifikatoriai "text" tipo.')

# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://www.sqlite.org/lang_createtable.html

db_file_name='auto_skelbimai_txt.db'
db_connect = sqlite3.connect(db_file_name)
cs = db_connect.cursor()
print(f"sqlite3.connect('{db_file_name}')")
print(dt.datetime.now().replace(microsecond=0))

Paskaita nr. 45, 2024-10-22
Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.
Nuo šios versijos parametrų identifikatoriai "text" tipo.
sqlite3.connect('auto_skelbimai_txt.db')
2024-10-24 16:01:21


In [67]:
# Pradedu kurti db klasę.
# https://stackoverflow.com/questions/38076220/python-mysqldb-connection-in-a-class
# https://stackoverflow.com/questions/75688682/how-should-i-type-my-sqlite-connection-and-cursor-in-a-python-class-before-i-pro
# https://pynative.com/python-sqlite/
# https://www.geeksforgeeks.org/python-sqlite-connecting-to-database/
# https://www.reddit.com/r/learnpython/comments/4lznux/best_technique_for_passing_sqlite3_connection/

print('Kol kas tik prisijungia db.')

import sqlite3

class things_with_parameters:
    production=False # 'True' for production, 'False' for testing.

    def __init__(self,name): # constructor.
        self._name=name
        self._connection = sqlite3.connect(name)
        self._cursor = self._connection.cursor()
        things_with_parameters.production or print(f"Connected '{name}'\n")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
    
    def __str__(self):

        rez=f'Database "{self._name}"\nThe structure:\n'
        for (tableName,) in self.query(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
        ):
            rez+="{}:\n".format(tableName)
            for (
                columnID, columnName, columnType,
                columnNotNull, columnDefault, columnPK,
            ) in self.query("pragma table_info('{}');".format(tableName)):
                rez+="  {id}: {name}({type}){null}{default}{pk}\n".format(
                    id=columnID,
                    name=columnName,
                    type=columnType,
                    null=" not null" if columnNotNull else "",
                    default=" [{}]".format(columnDefault) if columnDefault else "",
                    pk=" *{}".format(columnPK) if columnPK else "",
                )
        return rez

    @property
    def connection(self):
        return self._connection

    @property
    def cursor(self):
        return self._cursor

    def commit(self):
        self.connection.commit()

    def close(self, commit=True):
        if commit:
            self.commit()
        self.connection.close()
        things_with_parameters.production or print(f"Disconnected '{self._name}'")

    # def execute(self, sql, params=None):
    #     self._cursor.execute(sql, params or ())

    def execute(self, sql, params=None):
        if params is None:
            self._cursor.execute(sql)
        else:
            self._cursor.execute(sql, params)

    def fetchall(self):
        return self._cursor.fetchall()

    def fetchone(self):
        return self._cursor.fetchone()

    def query(self, sql, params=None):
        self._cursor.execute(sql, params or ())
        return self.fetchall()

Kol kas tik prisijungia db.


In [68]:
db_objektas=things_with_parameters(db_file_name)
print(db_objektas)
db_objektas.close()

Connected 'auto_skelbimai_txt.db'

Database "auto_skelbimai_txt.db"
The structure:
object_param:
  0: object_id(INTEGER) not null
  1: param_name(TEXT) not null
  2: param_value(TEXT)
  3: create_timestamp(INTEGER)
  4: update_timestamp(INTEGER)
object_x:
  0: id(INTEGER) *1
  1: href(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)
param_category:
  0: param_name(TEXT)
  1: param_description(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)

Disconnected 'auto_skelbimai_txt.db'


In [69]:
print("Išbandau funkciją 'printSchema(connection)'\n")

def printSchema(connection):
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

db_objektas=things_with_parameters(db_file_name)

connection=db_objektas.connection

printSchema(connection)

db_objektas.close()

Išbandau funkciją 'printSchema(connection)'

Connected 'auto_skelbimai_txt.db'

object_param:
  0: object_id(INTEGER) not null
  1: param_name(TEXT) not null
  2: param_value(TEXT)
  3: create_timestamp(INTEGER)
  4: update_timestamp(INTEGER)
object_x:
  0: id(INTEGER) *1
  1: href(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)
param_category:
  0: param_name(TEXT)
  1: param_description(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)
Disconnected 'auto_skelbimai_txt.db'


In [8]:
######################################################################
print(f"Kuriamos lentelės sqlite3 duomenųbazėje '{db_file_name}'") ###
######################################################################

sql_create_table = '''
create table if not exists object_x
(
    id integer primary key,
    href text,
    create_timestamp integer,
    update_timestamp integer
) strict
'''
cs.execute(sql_create_table)

sql_create_table = '''
create table if not exists object_param
(
    object_id integer not null,
    param_name text not null,
    param_value text,
    create_timestamp integer,
    update_timestamp integer,
    unique (object_id,param_name)
) strict
'''
cs.execute(sql_create_table)

sql_create_table = '''
create table if not exists param_category
(
    param_name text unique on conflict fail,
    param_description text,
    create_timestamp integer,
    update_timestamp integer
) strict
'''
cs.execute(sql_create_table)

print('Atlikta:',dt.datetime.now().replace(microsecond=0))
# https://www.sqlite.org/lang_createtable.html

Kuriamos lentelės sqlite3 duomenųbazėje 'auto_skelbimai_txt.db'
Atlikta: 2024-10-22 21:47:08


In [9]:
######################################################################
print('Perrašau lentelę į naują db')  ################################
######################################################################

sena_db = sqlite3.connect('Testas.db')
sena_cs = sena_db.cursor()

dabar=int(dt.datetime.today().timestamp()) # now(), today()
print('Dabar:',dabar,dt.datetime.fromtimestamp(dabar))

eilutės=[]
sena_cs.execute('select * from auto_skelbimai') # limit 9
for rezultatas in sena_cs.fetchall():
    eilutė=list(rezultatas)
    eilutė.append(dabar) 
    eilutė.append(dabar) 
    # print(eilutė)
    eilutės.append(eilutė)

sena_db.close()
print(f"Perskaityta {len(eilutės)} įrašų.")

nauja_db = sqlite3.connect(db_file_name) # current
nauja_cs = nauja_db.cursor()

sql_template='''INSERT OR REPLACE into object_x (id,href,create_timestamp,update_timestamp) values (?,?,?,?)'''
nauja_cs.executemany(sql_template,eilutės)

nauja_db.commit()
nauja_db.close()

Perrašau lentelę į naują db
Dabar: 1729622835
Perskaityta 1000 įrašų.
